In [99]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import nengo_dl
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, Conv3D
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, ShuffleSplit


In [100]:
# Dataset path is by default saved in dataset_result/bci_dataset.npz
dataset_path = os.path.join('dataset_result', 'bci_dataset.npz')

# Load the numpy file containing the dataset
dataset = np.load(dataset_path)
features, labels = dataset['features'], dataset['labels'] # get features and labels

print('Features shape:', features.shape)
print('Labels shape:', labels.shape)

Features shape: (2976, 14, 36, 10)
Labels shape: (2976,)


In [101]:
# Set seed for consistency
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

In [102]:
# Convert labels to one hot encoding
cat = OneHotEncoder()
labels = labels.reshape(-1, 1)
labels = cat.fit_transform(labels).toarray()
labels = labels.reshape((labels.shape[0], 1, -1))
print(labels)
print('Labels shape:', labels.shape)

# Reshape features for the NN
features = features.reshape((features.shape[0], 1, -1))
print('Features shape:', features.shape)

[[[0. 1.]]

 [[0. 1.]]

 [[0. 1.]]

 ...

 [[0. 1.]]

 [[0. 1.]]

 [[0. 1.]]]
Labels shape: (2976, 1, 2)
Features shape: (2976, 1, 5040)


In [103]:
def cnn_model():
    """
    Creates a CNN neural network
    :return: tensorflow model of the ANN
    """

    inp = Input(shape=(14, 360, 1), name='input_layer')
    conv2d = Conv2D(filters=9, kernel_size=(3, 3), activation='relu')(inp)
    dropout1 = Dropout(0.5, seed=seed)(conv2d)
    avg_pooling = AveragePooling2D(pool_size=(2, 2))(dropout1)
    flatten = Flatten()(avg_pooling)
    dense1 = Dense(100, activation='relu')(flatten)
    batch_norm = BatchNormalization()(dense1)
    dropout2 = Dropout(0.5, seed=seed)(batch_norm)
    output = Dense(2, activation='softmax', name='output_layer')(dropout2)

    return Model(inputs=inp, outputs=output)

In [104]:
def run_ann(model, train, valid, test, params_save_path, iteration, shuffle_training=True):
    x_train, y_train = train[0], train[1]
    x_valid, y_valid = valid[0], valid[1]
    x_test, y_test = test[0], test[1]

    converter = nengo_dl.Converter(model)

    with nengo_dl.Simulator(converter.net, minibatch_size=16) as simulator:
        simulator.compile(optimizer=keras.optimizers.Adam(),
                          loss=keras.losses.BinaryCrossentropy(),
                          metrics=['accuracy'])

        input_layer = converter.inputs[model.get_layer('input_layer')]
        output_layer = converter.outputs[model.get_layer('output_layer')]

        simulator.fit(
            x={ input_layer: x_train }, y={ output_layer: y_train },
            validation_data=(
                { input_layer:x_valid }, { output_layer: y_valid }
            ),
            epochs=10,
            shuffle=shuffle_training,
            callbacks=[EarlyStopping(patience=5, verbose=1, restore_best_weights=True)]
        )

        simulator.save_params(params_save_path)
        ann_eval = simulator.evaluate(x={ input_layer: x_test }, y={ output_layer: y_test }) # evaluate accuracy
        print('{}. ann accuracy: {:5f}%'.format(iteration, ann_eval['probe_accuracy'] * 100)) # log accuracy
        return ann_eval['probe_accuracy'] # return accuracy

In [105]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=seed, shuffle=True)
print('x_train shape: {}, y_train shape: {}', x_train.shape, y_train.shape)

# Create directory for ANN parameters used to construct the spiking network for Nengo
os.makedirs('nengo_params', exist_ok=True)
ann = []

num_iterations = 5 # Number of cross validation iterations
iteration = 1 # Current iteration
shuffle_split = ShuffleSplit(n_splits=num_iterations, test_size=0.25, random_state=seed) # Shuffle split for cross valid

for train, valid in shuffle_split.split(x_train):
    print('Current iteration: ', iteration)
    x_train_curr, y_train_curr = x_train[train], y_train[train]
    x_valid_curr, y_valid_curr = x_train[valid], y_train[valid]

    params_path = os.path.join('nengo_params', 'params_{}'.format(iteration))
    model = cnn_model()

    # run ann
    ann_result = run_ann(model=model,
                         train=(x_train_curr, y_train_curr),
                         valid=(x_valid_curr, y_valid_curr),
                         test=(x_test, y_test),
                         params_save_path=params_path,
                         iteration=iteration
                         )
    ann.append(ann_result)
    print(ann_result)

x_train shape: {}, y_train shape: {} (2232, 1, 5040) (2232, 1, 2)
Current iteration:  1
Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/10
104/104 [==============================] - 5s 24ms/step - loss: 0.9906 - probe_loss: 0.9906 - probe_accuracy: 0.5287 - val_loss: 1.0992 - val_probe_loss: 1.0992 - val_probe_accuracy: 0.5074

c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\converter.py:326: UserWarning: Layer type Dropout does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\converter.py:326: UserWarning: Cannot convert BatchNormalization layer to native Nengo objects unless inference_only=True or layer.trainable=False. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\converter.py:588: UserWarning: Activation type <function softmax at 0x000002777EED44C0> does not have a native Nengo equivalent; falling back to a TensorNode
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\simulator.py:460: UserWarning: No GPU support detected. See https://www.nengo.ai/nengo-dl/installation.html#installing-tensorflow for instructions on setting up TensorFlow with GPU support.
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\n